In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import stats
import numpy.ma as ma
import cartopy.crs as ccrs

#import mplotutils as mpu
#import cf_units
from tqdm import tqdm_notebook as tqdm

import glob



In [3]:
## functions to calculate trends, 

def linregress_grid(df,df_time,start,end):
    x,y=np.meshgrid(np.arange(36),np.arange(72))
    slope=np.full([36,72],np.nan)
    i_start=np.where(df_time==start)[0][0]
    i_end=np.where(df_time==end)[0][0]
    
    for i_x, i_y in zip(x.flatten(),y.flatten()):
        if not np.all(np.isnan(df[i_start:i_end+1,i_x,i_y])):
            #print(np.all(df[i_start:i_end+1,i_x,i_y]==np.nan),df[i_start:i_end+1,i_x,i_y])
            slope[i_x,i_y]=100*stats.linregress(np.arange(end-start+1)[~np.isnan(df[i_start:i_end+1,i_x,i_y])],
                                                df[i_start:i_end+1,i_x,i_y][~np.isnan(df[i_start:i_end+1,i_x,i_y])])[0]
    return slope

def mean_trend_from_dict(df,df_time,start,end, roll=False):
    
    print([df[key].shape for key in df.keys()])
    all_trends=np.stack([linregress_grid(df[key],df_time,start,end) for key in tqdm(df.keys())])
    
    if roll:
        return np.roll(np.mean(all_trends,axis=0),36,1)
    else:
        return np.mean(all_trends,axis=0)
    
def trend_dist_from_dict(df,df_time,start,end, roll=False):
    
    all_trends=np.stack([linregress_grid(df[key],df_time,start,end) for key in tqdm(df.keys())])
    
    if roll:
        return np.roll(np.quantile(all_trends,[0.05,0.95],axis=0),36,1)
    else:
        return np.quantile(all_trends,[0.05,0.95],axis=0)
    
def trend_from_dict(df,df_time,start,end, roll=False):
    
    all_trends=np.stack([linregress_grid(df[key],df_time,start,end) for key in tqdm(df.keys())])
    
    if roll:
        return np.roll(all_trends,36,1)
    else:
        return all_trends
        
            
def create_mask(df):
    
    ##First check that the year is 'valid' i.e. 5 months are non nana
    df=df.reshape(-1,12,df.shape[1],df.shape[2])
    valid_idx=np.zeros([df.shape[0],36,72])
    
    for yr_idx in range(df.shape[0]):
        valid_idx[yr_idx,:,:]=np.where(np.isnan(df[yr_idx,:,:,:]).sum(axis=0)>6,1,0)
        

    cutoff=valid_idx.shape[0]%5
            
    valid_idx=valid_idx[cutoff:,:,:].reshape(5,-1,36,72)
    
    mask_idx=np.zeros([5,36,72])
    
    for idx_block in range(5):
        mask_idx[idx_block,:,:]=np.where(valid_idx[idx_block,:,:,:].sum(axis=0)>np.floor(0.2*valid_idx.shape[1]),
                                                np.nan,1)
        
    print(valid_idx.shape,np.floor(0.8*valid_idx.shape[1]))
    mask_idx=np.prod(mask_idx,axis=0)
    
    return np.roll(~np.isnan(mask_idx),36,1)

In [1]:
# read in observations
hadcrut_anom={}

dir_var_hn='data/HadCRUT_observations/'
obs_name_list=sorted(glob.glob(dir_var_hn+'HadCRUT.4.6.0.0.anomalies.*.nc'))

for i,obs_name in enumerate(obs_name_list): 
    data=xr.open_mfdataset(obs_name).groupby('time.year').mean('time').roll(longitude=180, roll_coords=True).sel(year=slice('1869-01-01', '2101-01-01'))
    hadcrut_anom[i]=data.temperature_anomaly.values

# reference year to calculate trend    
ref=1951

hadcrut_med=xr.open_mfdataset('data/HadCRUT.4.6.0.0.median.nc')
hadcrut_med=hadcrut_med.groupby('time.year').mean('time')#.sel(time=slice('1870-01-01', '2101-01-01'))

trend_hadcrut_med={}
trend_hadcrut={}

# trends for all years in temperature
for year_idx in range(2000,2019):
 
    trend_hadcrut_med[year_idx]=linregress_grid(hadcrut_med.sel(year=slice(1870,2021)).roll(longitude=180, roll_coords=True).temperature_anomaly.values,
                         hadcrut_med.sel(year=slice(1870,2021)).year.values,ref,year_idx)
    trend_hadcrut[year_idx]=trend_dist_from_dict(hadcrut_anom,np.arange(1870,2021),ref,year_idx)

NameError: name 'glob' is not defined

In [ ]:
# match [temperature, years] to 5 degree grid, save as file
degrees = 5
lons = np.linspace(degrees*.5,360-degrees*.5,int(360/degrees))
lats = np.linspace(-90+degrees*.5,90-degrees*.5,int(180/degrees))
xv, yv = np.meshgrid(lons,lats)

temp = list()
years = list()
lon = list()
lat = list()

for year in range(2000,2019):
    temps = trend_hadcrut_med[year].flatten()
    temp.extend(temps)
    years.extend([year for i in range(len(temps))])
    lon.extend(list(xv.flatten()))
    lat.extend(list(yv.flatten()))
    
    
year_temp_grid = pd.DataFrame({"year": years, "temperature": temp, "LON_5":lon,"LAT_5":lat})
year_temp_grid.to_csv("data/grid5_temp_years.csv")

In [3]:
def weighted_avg_with_nan(data, weights):
    masked = np.ma.masked_array(data, np.isnan(data))
    select = np.invert(np.ma.getmaskarray(masked))
    data = data[select]
    weights = weights[select]
    return np.average(data, weights=weights)

In [5]:
# match fine subgrid with GDL code on coarse temperature-trend data
year_temp_grid = pd.read_csv("data/grid5_temp_years.csv")
grid_region = pd.read_csv('data/subgrid.csv')
grid_region["LON_5"] = grid_region.LON_5.apply(lambda x: x+360 if x<0 else x)
grid_region = grid_region[['LAT_5','LON_5','GDLcode', 'area']]
year_temp_grid = year_temp_grid.merge(grid_region, on=["LAT_5", "LON_5"], how="left")
region_temp = year_temp_grid.groupby(["GDLcode","year"])\
              .apply(lambda x: weighted_avg_with_nan(data=x.temperature, weights=x.area))\
              .reset_index().rename(columns={0:'temperature_trend'})
# add area data and calculate temp trend for each country
region_area = year_temp_grid.groupby(["GDLcode","year"])[["area"]].sum().reset_index()
region_area["CountryGDL"] = region_area["GDLcode"].apply(lambda x: x[:3]+'t')
region_temp = region_temp.merge(region_area, how="left", on=["GDLcode","year"])

country_temp = region_temp.groupby(["CountryGDL", "year"])\
              .apply(lambda x: weighted_avg_with_nan(data=x.temperature_trend, weights=x.area))\
              .reset_index().rename(columns={0:'temperature_trend'})
country_area =  region_temp.groupby(["CountryGDL", "year"])[["area"]].sum().reset_index()
country_temp = country_temp.merge(country_area, how="left", on=["CountryGDL","year"]).rename(columns={"CountryGDL":"GDLcode"})

region_country_temp = pd.concat([region_temp.drop(columns=["CountryGDL"]), country_temp])
region_country_temp.to_csv("data/region_temp_years.csv", index=False)


/tmp/ipykernel_64248/2078583284.py:3: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  grid_region = pd.read_csv('data/subgrid.csv')


In [ ]:
   
gen=6
usr_time_res='ann'
period=2016-1891
##create datasetsfor allforcing af, historical natural hn and preindustrial control piCONTROL
y_af={}
y_hn={}
y_pi={}




from loading import load_data_single_mod
import importlib
importlib.reload(sys.modules['loading'])
from loading import load_data_single_mod
#import glob
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")


gen=6
period=2018-1870

##create datasetsfor allforcing af, historical natural hn and preindustrial control piCONTROL
##models only usable if all-forcing run exists, at least 3 histNat runs exist and length of histNat is same as period under investigation

y_af={}
y_hn={}
y_pi={}

time={}

models=['MIROC6', 'IPSL-CM6A-LR', 'CanESM5', 'HadGEM3-GC31-LL', 'CNRM-CM6-1', 'GFDL-ESM4', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'NorESM2-LM', 'MRI-ESM2-0']

usable_models=[]
for model in models:
    
    scenario='historical'

    
    if gen==6:
        member = 'r*i1p1f*'
        hist_name = 'hist-nat'
    elif gen==5:
        hist_name = 'historicalNat'
        member = 'r*i1p1'
    
    len_hn=[]
    len_pi=[]
    
 
    y_af[model],time[model],srex,srex_names,lon_pc,lat_pc,idx_l,wgt_l=load_data_single_mod(gen,model,scenario,Tanglob_idx=False,Tref_all=True,Tref_start='1961-01-01',Tref_end='1991-01-01')
    len_af=[y_af[model][key].shape[0] for key in y_af[model].keys()]
    
        
    dir_var_hn='/net/so4/landclim/snath/DA/data/%s/regridded/'%model
    run_name_hn_list=sorted(glob.glob(dir_var_hn+'tas_ann_'+model+'_'+hist_name+'_'+member+'_g05.nc'))

In [ ]:
((((seagrass OR mangrove*) AND  (deforest* OR afforest* OR conserv* OR restor* OR manag*) )OR "blue carbon")  AND ((carbon OR CO2) NEAR/3 (sequest* OR accumulat* OR storage OR capture)))

## Some plots

In [ ]:


## create mask based on criteria 

hadcrut_med=xr.open_mfdataset('data/HadCRUT.4.6.0.0.median.nc').sel(time=slice('1951-01-01', '2019-01-01'))
mask_1951=create_mask(hadcrut_med.temperature_anomaly.values)


##Example plots of trends
n_col = 2
n_row = 3

fs_title=16

fig=plt.figure(figsize=(n_col*13, n_row * 11))
plt.rcParams.update({'font.size': 12})
plt.rcParams.update({'mathtext.default':'regular'}) 
plt.rcParams.update({'mathtext.default':'it'}) 


grid = plt.GridSpec(n_row*3, n_col*9+3, wspace=1, hspace=0) # create a grid for the subplots #0.12

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
lons=hadcrut_med.longitude.values
lats=hadcrut_med.latitude.values

i=0
for ref in [1951]:
    
    i_y=0

    ax = plt.subplot(grid[i+1:i+2,i_y*15:i_y*15+15], projection=ccrs.PlateCarree(central_longitude=180))

    i_y+=1
    i+=1
    
    y_ma = np.zeros(mask_1951.shape)
    y_ma = ma.masked_array(y_ma, mask=mask_1951==False)
    y_ma[mask_1951]=trend_hadcrut_med[2000][mask_1951]

    ax.coastlines()
    mesh_1=ax.pcolormesh(lons, lats, y_ma, transform=ccrs.PlateCarree(central_longitude=180), cmap=plt.cm.get_cmap('coolwarm',14),vmin=-5,vmax=5,rasterized=True)
    ax.set_title("HadCRUT4 %i-2000"%(ref),y=1.02,fontsize=12)
    cmap_mesh=mesh_1.get_cmap()
    cmap_mesh.set_bad('gray')
    mesh_1.set_cmap(cmap_mesh)
    
    ax = plt.subplot(grid[i-(i%2)+1:i-(i%2)+2,i_y*15:i_y*15+15], projection=ccrs.PlateCarree(central_longitude=180))
    i_y+=1
    
    i+=1
    
    ax.coastlines()
    mesh_1=ax.pcolormesh(lons, lats, trend_hadcrut_med[2018], transform=ccrs.PlateCarree(central_longitude=180), cmap=plt.cm.get_cmap('coolwarm',14),vmin=-5,vmax=5,rasterized=True)
    ax.set_title("HadCRUT4 %i-2018"%(ref),y=1.02,fontsize=12)
    cmap_mesh=mesh_1.get_cmap()
    cmap_mesh.set_bad('gray')
    mesh_1.set_cmap(cmap_mesh)

    
axcbar = plt.subplot(grid[i-(i%2):i-(i%2)+1,5:25])
cbar=plt.colorbar(mesh_1,orientation='horizontal',fraction=0.4,aspect=45)
cbar.set_label('Temperature Trends °C/Century')  
plt.axis('off')   


In [ ]:
# gif for grid

from cartopy import crs as ccrs
import cartopy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import imageio

def temp_trend_grid(year, grid_df):
    df = grid_df.loc[grid_df.year==year]
    fig = plt.figure(dpi=150, figsize=(7,4.5))

    ax = plt.subplot(projection=ccrs.EqualEarth())

    ax.coastlines(lw=0.1)

    n = np.array(df.temperature).reshape(len(df.LAT_5.unique()),len(df.LON_5.unique()))

    mesh = ax.pcolormesh(
        df.LON_5.unique(),
        df.LAT_5.unique(),
        n,
        alpha=1,
        transform=ccrs.PlateCarree(),
        norm = mpl.colors.Normalize(vmin=-5,vmax=5),
        cmap="RdBu_r"
    )

    cbar = plt.colorbar(mesh, orientation="horizontal",
                       fraction=0.046, pad=0.04)
    cbar.set_label('Temperature difference compared to 1951')

    ax.set_title(f'Temperature Trend - Year {year}')
    fig.savefig(f"figures/grid5_temptrend1951_year{year}.png")
    plt.close()
    
    
grid_df = pd.read_csv("data/grid5_temp_years.csv")
for i in range(2000,2019):
    temp_trend_grid(i, grid_df)

filenames = [f"figures/grid5_temptrend1951_year{year}.png" for year in range(2000,2019)]
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('grid5_temperature.gif', images, duration=0.5 )

In [ ]:
# gif for region
import geoplot
import matplotlib.pyplot as plt
import matplotlib as mpl
import geopandas
from datetime import datetime

def plot_region_temp(year, dat):
    df = dat.loc[dat.year==year]
    
    fig, ax = plt.subplots(dpi=150, figsize=(7,4.5))
    ax = plt.subplot(projection=ccrs.EqualEarth())

    ax.coastlines(lw=0.1)

    cmap = mpl.cm.get_cmap('RdBu_r')
    norm = mpl.colors.Normalize(vmin=-5,vmax=5)

    #ax.add_geometries(merged.geometry, lw=0.1, linestyle=':',crs=ccrs.EqualEarth(),color=colors)
    for i, row in df.iterrows():

        ax.add_geometries(
            [row['geometry']],color=cmap(norm(row['temperature_trend'])), 
            crs=ccrs.PlateCarree(),lw=0.1, linestyle=':',ec="black"
        )    
    cbar = fig.colorbar(
        mpl.cm.ScalarMappable(norm=norm, cmap=cmap), 
        ax=ax, orientation="horizontal",
        fraction=0.046, pad=0.04
    )
    ax.set_title(f'Temperature Trend - Year {year}')
    cbar.set_label('Temperature difference compared to 1951')
    fig.savefig(f"figures/region_temptrend1951_year{year}.png")
    plt.close()

gdl_shapes = geopandas.read_file("data/GDL Shapefiles V5 11-21.shp")
reg_df = pd.read_csv("data/region_temp_years.csv").merge(gdl_shapes,left_on="GDLcode",right_on="gdlcode").fillna(0)
for year in range(2000,2019):
    print(year)
    current_time = datetime.now().strftime("%H:%M:%S")
    print("Current Time =", current_time)
    plot_region_temp(year, reg_df)
    

In [ ]:
filenames = [f"figures/region_temptrend1951_year{year}.png" for year in range(2000,2018)]
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('region_temperature.gif', images, duration=0.5 )

In [ ]:
from cartopy import crs as ccrs
import cartopy
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(dpi=150, figsize=(7,4.5))

ax = plt.subplot(projection=ccrs.EqualEarth())

ax.coastlines(lw=0.1)

n = np.array(grid_df.temp).reshape(len(grid_df.LAT.unique()),len(grid_df.LON.unique()))

mesh = ax.pcolormesh(
    grid_df.LON.unique(),
    grid_df.LAT.unique(),
    n,
    alpha=1,
    transform=ccrs.PlateCarree(),
    norm = mpl.colors.Normalize(vmin=-5,vmax=5),
    cmap="RdBu_r"
)

cbar = plt.colorbar(mesh, orientation="horizontal",
                   fraction=0.046, pad=0.04)
cbar.set_label('area ($km^2$)')

ax.set_title('2.5 degree grid cells')